In [ ]:
%pip install augini

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import json
import os
import sys
sys.path.append('..')
from augini import Augini

# Input your api key
def get_api_key():
    api_key = os.getenv('API_KEY')
    if api_key:
        print("Using API key from environment variable.")
        return api_key
    else:
        api_key = input("Enter your API key manually: ")
        return api_key

# Set up Augini
api_key = get_api_key()
augini = Augini(api_key=api_key, use_openrouter=True, model='openai/gpt-4o-mini-2024-07-18')

In [ ]:
# Create a sample customer dataset
np.random.seed(42)
n_customers = 100

data = {
    'CustomerID': [f'C{i:04d}' for i in range(1, n_customers + 1)],
    'Age': np.random.randint(18, 80, n_customers),
    'Tenure': np.random.randint(0, 10, n_customers),
    'MonthlyCharges': np.random.uniform(20, 200, n_customers).round(2),
    'TotalCharges': np.random.uniform(100, 5000, n_customers).round(2),
    'Contract': np.random.choice(['Month-to-month', 'One year', 'Two year'], n_customers),
    'PaymentMethod': np.random.choice(['Electronic check', 'Mailed check', 'Bank transfer', 'Credit card'], n_customers),
    'Churn': np.random.choice([0, 1], n_customers, p=[0.7, 0.3])  # 30% churn rate
}

df = pd.DataFrame(data)

print("Original Dataset:")
display(df.head())

# 1. Data Augmentation: Add synthetic features using Augini

In [ ]:
augment_prompt = """
Based on the customer's age, tenure, monthly charges, total charges, contract type, and payment method, suggest:
1. A likely reason for churn (if applicable)
2. A personalized retention offer
3. The customer's estimated lifetime value (in dollars)

Respond with a JSON object with keys 'ChurnReason', 'RetentionOffer', and 'EstimatedLTV'.
"""

augmented_df = augini.augment_columns(df, ['ChurnReason', 'RetentionOffer', 'EstimatedLTV'], custom_prompt=augment_prompt)

print("\nAugmented Dataset:")
display(augmented_df.head())

In [ ]:
# After augmenting the dataset
print("\nExample Augmented Data:")
for i in range(3):
    customer = augmented_df.iloc[i]
    print(f"\nCustomer ID: {customer['CustomerID']}")
    print(f"Age: {customer['Age']}, Tenure: {customer['Tenure']}, Monthly Charges: ${customer['MonthlyCharges']:.2f}")
    print(f"Contract: {customer['Contract']}, Churn: {'Yes' if customer['Churn'] else 'No'}")
    print(f"Churn Reason: {customer['ChurnReason']}")
    print(f"Retention Offer: {customer['RetentionOffer']}")
    print(f"Estimated LTV: ${customer['EstimatedLTV']:.2f}")

Key Observations:

- The **augini** generates personalized retention offers based on each customer's specific situation, considering factors like age, contract type, and perceived churn risk.
- Estimated LTV varies significantly, highlighting the importance of tailored retention strategies for high-value customers.
- The **augini** identifies potential churn reasons even for non-churned customers, allowing for proactive retention efforts.
- The retention offers are designed to address the specific concerns or risk factors identified for each customer.

# 2. Data Analysis and Visualization

In [ ]:
plt.figure(figsize=(16, 12))

# Churn rate by contract type
plt.subplot(2, 2, 1)
sns.countplot(data=df, x='Contract', hue='Churn')
plt.title('Churn Rate by Contract Type')

# Monthly charges distribution for churned vs non-churned customers
plt.subplot(2, 2, 2)
sns.histplot(data=df, x='MonthlyCharges', hue='Churn', kde=True)
plt.title('Monthly Charges Distribution')

# Tenure vs Churn
plt.subplot(2, 2, 3)
sns.boxplot(data=df, x='Churn', y='Tenure')
plt.title('Tenure vs Churn')

# Correlation heatmap
plt.subplot(2, 2, 4)
corr_matrix = df[['Age', 'Tenure', 'MonthlyCharges', 'TotalCharges', 'Churn']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Correlation Heatmap')

plt.tight_layout()
plt.show()

# 3. Churn Prediction Model

In [ ]:
features = ['Age', 'Tenure', 'MonthlyCharges', 'TotalCharges']
X = df[features]
y = df['Churn']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("\nChurn Prediction Model Performance:")
print(classification_report(y_test, y_pred))

# 4. Feature Importance
feature_importance = pd.DataFrame({
    'feature': features,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x='importance', y='feature', data=feature_importance)
plt.title('Feature Importance for Churn Prediction')
plt.show()



# 5. Insights Generation using Augini

In [8]:
insights_prompt = """
Based on the customer churn analysis, provide three key insights about customer behavior, churn patterns, and potential retention strategies.
Consider factors such as contract types, payment methods, tenure, and charges.

Respond with a JSON object with keys 'Insight1', 'Insight2', and 'Insight3'.
"""

# Use augment_single with the entire DataFrame
insights = augini.augment_single(augmented_df.head(4), 'Insights', custom_prompt=insights_prompt)

In [ ]:
insights.head()

In [ ]:
print("Insights from the first 3 rows:")
for i in range(3):
    insight = insights['Insights'].iloc[i]
    print(f"\nRow {i + 1}:")
    if isinstance(insight, dict):
        for key, value in insight.items():
            print(f"{key}: {value}")
    else:
        print("Unexpected data type. Raw insight:")
        print(insight)
    print("-" * 50)  # Separator between rows

# 6. Retention Strategy Recommendations

In [11]:
strategy_prompt = """
Based on the churn analysis and generated insights, provide three strategic recommendations to reduce customer churn and improve retention.
Consider personalized offers, pricing strategies, contract adjustments, and customer engagement initiatives.

Respond with a JSON object with keys 'Strategy1', 'Strategy2', 'Strategy3'.
"""

strategies = augini.augment_single(augmented_df.head(4), 'Strategies', custom_prompt=strategy_prompt)

In [ ]:
# Assuming your DataFrame is named 'insights'
print("Strategies from the first 3 rows:")
for i in range(3):
    insight = strategies['Strategies'].iloc[i]
    print(f"\nRow {i + 1}:")
    if isinstance(insight, dict):
        for key, value in insight.items():
            print(f"{key}: {value}")
    else:
        print("Unexpected data type. Raw insight:")
        print(insight)
    print("-" * 50)  # Separator between rows

# Summary of Retention Strategies

## Strategy 1: Personalized Offers and Loyalty Programs

- **Long-term Customers**: Implement a personalized loyalty discount program, offering up to 15% off monthly charges for customers with 9+ years tenure.
- **Age-based Targeting**: Tailor offers based on age demographics, particularly for senior customers.
- **Contract Commitment**: Encourage month-to-month customers to switch to longer-term plans with incentives like discounted rates (e.g., $54.44/month for 6 months on a 12-month contract).

## Strategy 2: Flexible and Tiered Pricing

- **Usage-based Plans**: Introduce flexible pricing options that align with different usage patterns and customer needs.
- **Senior-friendly Options**: Develop tiered pricing that caters specifically to senior citizens' budgets and service requirements.
- **Competitive Alignment**: Analyze the market to ensure prices reflect perceived value and introduce loyalty rewards for long-term customers.

## Strategy 3: Enhanced Customer Engagement

- **Proactive Communication**: Establish regular check-ins with customers to address concerns and offer personalized solutions.
- **Feedback Mechanism**: Implement quarterly customer satisfaction surveys to gather insights and preemptively address issues.
- **Relationship Building**: Foster stronger connections through ongoing engagement initiatives and responsive customer service.

These strategies focus on personalizing offers, creating flexible pricing structures, and improving customer engagement to reduce churn and enhance overall customer satisfaction.